In [0]:
import pandas as pd

In [0]:
df = spark.table("tsla")
df.show()

+---+----------+------------------+------------------+------------------+------------------+---------+
|_c0|      Date|              Open|              High|               Low|             Close|   Volume|
+---+----------+------------------+------------------+------------------+------------------+---------+
|  0|2010-06-29|1.2666670083999634|1.6666669845581055|1.1693329811096191|1.5926669836044312|281494500|
|  1|2010-06-30|1.7193330526351929|2.0280001163482666| 1.553333044052124|1.5886670351028442|257806500|
|  2|2010-07-01|1.6666669845581055|1.7280000448226929|1.3513330221176147|1.4639999866485596|123282000|
|  3|2010-07-02|1.5333329439163208|1.5399999618530273|  1.24733304977417|1.2799999713897705| 77097000|
|  4|2010-07-06|1.3333330154418945|1.3333330154418945|1.0553330183029175|1.0740000009536743|103003500|
|  5|2010-07-07|1.0933330059051514| 1.108667016029358|0.9986670017242432| 1.053333044052124|103825500|
|  6|2010-07-08|1.0759999752044678|1.1679999828338623|1.0379999876022339|

In [0]:
# Convert spark df to pandas df
tesla_df = df.toPandas()

# Previewing the first few records of the tesla_df to note the earliest stock date available.
tesla_df.head(5)

,_c0,Date,Open,High,Low,Close,Volume
0,0,2010-06-29,1.2666670083999634,1.6666669845581055,1.1693329811096191,1.5926669836044312,281494500
1,1,2010-06-30,1.7193330526351929,2.0280001163482666,1.553333044052124,1.5886670351028442,257806500
2,2,2010-07-01,1.6666669845581055,1.7280000448226929,1.3513330221176147,1.4639999866485596,123282000
3,3,2010-07-02,1.5333329439163208,1.5399999618530273,1.24733304977417,1.2799999713897705,77097000
4,4,2010-07-06,1.3333330154418945,1.3333330154418945,1.0553330183029175,1.0740000009536743,103003500


In [0]:
# Previewing the last few records of the tesla_df to note the latest stock date available.
tesla_df.tail()

,_c0,Date,Open,High,Low,Close,Volume
3529,3529,2024-07-09,251.0,265.6099853515625,250.3000030517578,262.3299865722656,160210900
3530,3530,2024-07-10,262.79998779296875,267.5899963378906,257.8599853515625,263.260009765625,128519400
3531,3531,2024-07-11,263.29998779296875,271.0,239.64999389648438,241.02999877929688,221707300
3532,3532,2024-07-12,235.8000030517578,251.83999633789062,233.08999633789062,248.22999572753906,155694400
3533,3533,2024-07-15,255.96499633789062,265.5799865722656,251.72999572753906,252.63999938964844,142831728


In [0]:
# Check for Duplicates

duplicates = tesla_df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")

# Drop duplicates if any
tesla_df = tesla_df.drop_duplicates()
print("Duplicates Drop Complete")

Number of duplicate rows: 0
Duplicates Drop Complete


In [0]:
# Check for Missing Values

missing = tesla_df.isnull().sum()
print("\n--- Missing Values ---")
print(missing[missing > 0])

# Drop rows with any missing values (or handle as needed)
tesla_df = tesla_df.dropna()


--- Missing Values ---
Series([], dtype: int64)


In [0]:
# COMMAND ----------
# Convert Date column to datetime and sort

tesla_df['Date'] = pd.to_datetime(tesla_df['Date'], errors='coerce')
tesla_df = tesla_df.dropna(subset=['Date'])  # remove rows where date couldn't be parsed
tesla_df = tesla_df.sort_values('Date')

In [0]:
num_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

# Convert columns to numeric, coercing errors to NaN (so bad strings become NaN)
for col in num_cols:
    tesla_df[col] = pd.to_numeric(tesla_df[col], errors='coerce')

# Now you can safely check for negatives, ignoring NaNs
for col in num_cols:
    if (tesla_df[col] < 0).any():
        print(f"Warning: Negative values found in {col}")

# Drop rows with NaN in numeric columns (optional, since those came from bad data)
tesla_df = tesla_df.dropna(subset=num_cols)

# Now filter out negative values
tesla_df = tesla_df[(tesla_df[num_cols] >= 0).all(axis=1)]

In [0]:
# Print Final Stats

print(f"\nFinal cleaned shape: {tesla_df.shape}")
print(tesla_df.describe())


Final cleaned shape: (3534, 7)
              Open         High          Low        Close        Volume
count  3534.000000  3534.000000  3534.000000  3534.000000  3.534000e+03
mean     74.997811    76.642865    73.249666    74.985912  9.694511e+07
std     102.281381   104.556354    99.811577   102.217353  7.877581e+07
min       1.076000     1.108667     0.998667     1.053333  1.777500e+06
25%      11.448667    11.753834    11.191166    11.532334  4.749600e+07
50%      17.417334    17.645333    17.064000    17.400333  8.216325e+07
75%     159.177502   161.312500   153.632500   158.180004  1.228650e+08
max     411.470001   414.496674   405.666656   409.970001  9.140820e+08


In [0]:
# Convert pandas DataFrame to Spark DataFrame
tesla_sdf = spark.createDataFrame(tesla_df)

# Save as Delta table (or use Parquet/CSV)
tesla_sdf.write.mode("overwrite").format("delta").save("/mnt/data/tesla_delta")